# NHL Game Predictor
###### Note, file is a bit outdated, as the file was overwritten by a pull, but wasn't stashed or saved anywhere
###### Difference was better models, Confusion Matrix, updated sort method 
### Important Notes
* When I use the year, it refers to the year the season ends.
    * Like for example the 2024-2025 is marked as 2025.
* nhl-XX.csv is the score data for a given season.
    * Links to get this data is as follows:
      * [2022 Season](https://shanemcd.org/2021/09/23/2021-22-nhl-schedule-and-results-in-excel-xlsx-and-csv-formats/)
      * [2023 Season](https://shanemcd.org/2022/07/12/2022-23-nhl-schedule-and-results-in-excel-xlsx-and-csv-formats/)
      * [2024 Season](https://shanemcd.org/2023/08/23/2023-24-nhl-schedule-and-results-in-excel-xlsx-and-csv-formats/)
      * [2025 Season](https://shanemcd.org/2024/09/07/2024-25-nhl-schedule-and-results-in-excel-xlsx-and-csv-formats/)
      * All on shanemcd.org, note that I have to redownload the 2025 season data for updated results (csv download of course).
* teamsXX.csv is the team data for a given season.
    * Links to the data is from [MoneyPuck.com](https://moneypuck.com/data.htm)
    * Downloads are from the Team Level

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import scale, MinMaxScaler, StandardScaler
from knn import KNN
pd.set_option('display.max_columns', None)

### getTeam(abbreviation)
This returns the team name given the abbrevation

In [3]:
abbrev = pd.read_csv("abbreviations.csv")
abbrev.index = abbrev["Abbrev"]
abbrev = abbrev.drop("Abbrev", axis=1)
abbrev = abbrev.to_dict("index")
abbrev = {k: v['Team'] for k, v in abbrev.items()}
abbrev
def getTeam(text):
    text = text.upper()
    return abbrev[text]

The cell bellow imports the game data and makes only gets the score and winner

In [4]:
def import_games(file):
    game_data = pd.read_csv(file)
    game_data = game_data[game_data["Status"] != "Scheduled"]
    game_data = game_data.rename(columns={"Score": "AwayScore", "Score.1": "HomeScore","Visitor": "Away"})
    game_data["HomeWin"] = game_data["HomeScore"] > game_data["AwayScore"]
    game_data["HomeWin"] = game_data["HomeWin"].astype(int)
    game_data = game_data[["Away","AwayScore","Home","HomeScore","HomeWin"]]
    return game_data
games22 = import_games("nhl-22.csv")
games23 = import_games("nhl-23.csv")
games24 = import_games("nhl-24.csv")
games25 = import_games("nhl-25.csv")
games25

,Away,AwayScore,Home,HomeScore,HomeWin
0,New Jersey Devils,4.0,Buffalo Sabres,1.0,0
1,Buffalo Sabres,1.0,New Jersey Devils,3.0,1
2,St. Louis Blues,3.0,Seattle Kraken,2.0,0
3,Boston Bruins,4.0,Florida Panthers,6.0,1
4,Chicago Blackhawks,2.0,Utah Hockey Club,5.0,1
...,...,...,...,...,...
418,Seattle Kraken,2.0,New Jersey Devils,3.0,1
419,Pittsburgh Penguins,2.0,New York Rangers,4.0,1
420,Columbus Blue Jackets,2.0,Vancouver Canucks,5.0,1
421,Dallas Stars,2.0,Vegas Golden Knights,3.0,1


In [5]:
def get_records(data):
    results = {}
    for i, row in data.iterrows():
        away = row["Away"]
        awayScore = row["AwayScore"]
        home = row["Home"]
        homeScore = row["HomeScore"]
        homeWin = row["HomeWin"]
        # [points scored, points allowed, wins, gp, win%]
        if away not in results:
            results[away] = [0,0,0,0,0]
        if home not in results:
            results[home] = [0,0,0,0,0]
        results[away][3] += 1
        results[home][3] += 1
        if homeWin == 0:
            results[away][2] += 1
        if homeWin == 1:
            results[home][2] += 1
        results[away][0] += awayScore
        results[away][1] += homeScore
        results[home][0] += homeScore
        results[home][1] += awayScore
        results[away][4] = results[away][2] / results[away][3]
        results[home][4] = results[home][2] / results[home][3]
        df = pd.DataFrame.from_dict(results, orient="index", columns=["GS", "GA", "Wins","GP","Win%"])
        df["GD"] = df["GS"] - df["GA"]
        df = df[["GS","GA","GD","Win%"]]
    return df
records22 = get_records(games22)
records23 = get_records(games23)
records24 = get_records(games24)
records25 = get_records(games25)
records25

,GS,GA,GD,Win%
New Jersey Devils,102.0,78.0,24.0,0.620690
Buffalo Sabres,78.0,81.0,-3.0,0.423077
St. Louis Blues,71.0,83.0,-12.0,0.481481
Seattle Kraken,82.0,83.0,-1.0,0.464286
Boston Bruins,71.0,84.0,-13.0,0.500000
Florida Panthers,103.0,90.0,13.0,0.592593
Chicago Blackhawks,64.0,82.0,-18.0,0.307692
Utah Hockey Club,70.0,75.0,-5.0,0.400000
Toronto Maple Leafs,78.0,66.0,12.0,0.615385
Montreal Canadiens,73.0,94.0,-21.0,0.384615


In [6]:
def import_teams(file):
    season_data = pd.read_csv(file)
    season_data = season_data[season_data.situation == "all"]
    season_data = season_data.drop(["team.1", "position", "situation", "season", "name"], axis=1)
    season_data["team"] = season_data["team"].apply(getTeam)
    season_data = season_data.reset_index()
    season_data = season_data.drop(["index"], axis=1)
    return season_data

In [7]:
def add_records(teamdata, recorddata):
    teamdata.insert(1,"Win%",0.0)
    teamdata.insert(1,"GD",0)
    teamdata.insert(1,"GA",0)
    teamdata.insert(1,"GS",0)
    for i in recorddata.index:
        current = recorddata.loc[i]
        teamdata.loc[teamdata["team"] == i,"GS"] = current["GS"]
        teamdata.loc[teamdata["team"] == i,"GA"] = current["GA"]
        teamdata.loc[teamdata["team"] == i,"GD"] = current["GD"]
        teamdata.loc[teamdata["team"] == i,"Win%"] = current["Win%"]
    return teamdata

teamdata22 = import_teams("teams22.csv")
teamdata23 = import_teams("teams23.csv")
teamdata24 = import_teams("teams24.csv")
teamdata25 = import_teams("teams25.csv")
add_records(teamdata22, records22)
add_records(teamdata23, records23)
add_records(teamdata24, records24)
add_records(teamdata25, records25)

,team,GS,GA,GD,Win%,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst
0,Washington Capitals,105,72,33,0.692308,26,0.55,0.50,0.51,94212.0,835.84,96.40,55.50,203.62,28.54,438.53,323.41,91.00,97.19,91.82,750.0,396.0,442.0,1588.0,105.0,90.0,8.0,150.0,1.0,325.0,475.0,645.0,1041.0,97.0,228.0,765.0,535.0,118.0,397.0,787.0,237.0,122.0,22.88,30.39,43.13,39.0,29.0,37.0,1620.90,1146.0,1168.21,196.0,12.40,19.97,19.97,88.83,89.69,86.57,798.32,77.73,52.99,194.34,26.53,426.06,309.35,73.65,77.37,73.33,724.0,363.0,500.0,1587.0,72.0,73.0,7.0,156.0,1.0,356.0,429.0,652.0,1015.0,97.0,222.0,774.0,528.0,121.0,428.0,794.0,204.0,89.0,22.41,25.78,29.55,29.0,18.0,25.0,1559.50,1087.0,1074.94,209.0,11.90,15.39,15.39,74.24,73.86,71.87
1,Buffalo Sabres,78,81,-3,0.423077,26,0.49,0.52,0.51,94809.0,834.50,79.10,56.28,205.61,28.45,455.52,321.05,75.14,78.89,74.94,743.0,403.0,478.0,1624.0,77.0,84.0,12.0,156.0,5.0,359.0,465.0,666.0,1069.0,107.0,232.0,771.0,530.0,129.0,379.0,855.0,211.0,80.0,25.28,25.86,27.96,33.0,27.0,17.0,1620.59,1146.0,1142.58,180.0,12.62,19.09,19.09,72.62,72.42,70.31,795.95,81.36,53.61,191.34,26.85,428.49,313.36,78.52,81.49,78.68,753.0,342.0,405.0,1500.0,81.0,88.0,17.0,156.0,4.0,336.0,430.0,672.0,1014.0,104.0,222.0,797.0,552.0,138.0,391.0,807.0,199.0,89.0,23.17,25.32,32.87,23.0,29.0,29.0,1500.96,1095.0,1098.16,194.0,11.93,16.05,16.75,76.53,76.78,75.06
2,Vegas Golden Knights,99,82,17,0.642857,28,0.50,0.47,0.48,101847.0,843.68,88.37,55.91,197.90,27.68,453.98,329.16,82.24,88.79,82.67,834.0,319.0,435.0,1588.0,98.0,77.0,13.0,172.0,1.0,389.0,416.0,736.0,1055.0,64.0,142.0,815.0,540.0,154.0,451.0,827.0,223.0,103.0,24.36,27.49,36.51,34.0,36.0,28.0,1598.83,1153.0,1158.62,221.0,12.31,15.81,15.81,84.86,85.27,80.89,922.90,88.85,61.27,229.96,31.68,496.20,358.04,84.33,88.93,84.42,805.0,461.0,525.0,1791.0,81.0,108.0,11.0,164.0,5.0,464.0,444.0,7

In [8]:
def set_game_stats(games, teams):
    gameStats = games.copy(deep = False)
    difflist = []
    for i, row in gameStats.iterrows():
        awayStats = teams[teams["team"] == row["Away"]]
        homeStats = teams[teams["team"] == row["Home"]]
        awayStats = awayStats.drop(["team"], axis=1)
        homeStats = homeStats.drop(["team"], axis=1)
        diff = pd.DataFrame(np.subtract(homeStats.values, awayStats.values), columns=homeStats.columns)
        diff.index = [i]
        difflist.append(diff)
    difflist = pd.concat(difflist, ignore_index = False)
    difflist = pd.concat([gameStats, difflist], axis=1)
    return difflist
gameStats22 = set_game_stats(games22, teamdata22)
gameStats23 = set_game_stats(games23, teamdata23)
gameStats24 = set_game_stats(games24, teamdata24)
gameStats25 = set_game_stats(games25, teamdata25)
all_game_stats = pd.concat([gameStats22, gameStats23, gameStats24, gameStats25], ignore_index=True)
all_game_stats

,Away,AwayScore,Home,HomeScore,HomeWin,GS,GA,GD,Win%,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst
0,Pittsburgh Penguins,6.0,Tampa Bay Lightning,2.0,0,15.0,4.0,11.0,0.060976,0.0,-0.01,-0.01,-0.02,-272.0,-168.23,-3.21,-5.49,-62.17,-5.91,-85.68,-85.55,0.15,-3.24,0.11,-315.0,67.0,-95.0,-343.0,16.0,-63.0,-5.0,10.0,15.0,-78.0,-149.0,-331.0,-264.0,86.0,223.0,-21.0,114.0,-81.0,-186.0,-307.0,61.0,-2.0,-6.65,7.54,-4.09,10.0,-6.0,12.0,-350.10,-248.0,-254.12,-45.0,-0.70,-11.17,-12.03,8.12,7.68,7.84,-53.08,12.50,3.60,-36.59,-3.81,-3.49,-33.20,12.30,12.60,12.40,-135.0,74.0,-50.0,-111.0,6.0,-52.0,9.0,-29.0,10.0,-51.0,55.0,-141.0,-67.0,57.0,183.0,71.0,-119.0,-35.0,-121.0,-146.0,87.0,-2.0,1.78,10.97,-0.26,-4.0,1.0,9.0,-112.61,-61.0,-59.75,-28.0,0.99,-8.72,-8.35,21.83,21.78,20.56
1,Seattle Kraken,3.0,Vegas Golden Knights,4.0,1,50.0,-37.0,87.0,0.195122,0.0,0.04,0.01,0.03,1796.0,382.29,52.96,26.47,95.61,11.89,166.38,152.71,47.87,55.00,49.86,449.0,57.0,130.0,636.0,49.0,69.0,7.0,84.0,32.0,253.0,19.0,400.0,457.0,-17.0,-60.0,194.0,-91.0,160.0,90.0,291.0,144.0,71.0,8.66,17.78,26.50,2.0,28.0,19.0,715.96,506.0,548.29,18.0,5.72,16.22,16.21,42.46,44.81,42.57,50.43,9.21,6.04,6.00,0.76,23.01,19.00,7.62,6.44,4.94,92.0,-28.0,279.0,343.0,-40.0,15.0,2.0,-117.0,3.0,116.0,87.0,132.0,104.0,-16.0,-49.0,47.0,646.0,153.0,-10.0,-18.0,53.0,29.0,-0.41,8.02,1.58,-4.0,-9.0,-27.0,248.60,64.0,4.58,-37.0,1.53,0.24,0.28,10.46,7.22,6.01
2,Montreal Canadiens,1.0,Toronto Maple Leafs,2.0,1,94.0,-66.0,160.0,0.390244,0.0,0.12,0.07,0.08,-223.0,501.11,78.21,43.15,84.25,14.06,278.77,172.58,73.52,82.36,77.55,391.0,280.0,80.0,751.0,94.0,19.0,9.0,101.0,32.0,271.0,154.0,297.0,577.0,-46.0,-102.0,314.0,-258.0,229.0,-179.0,269.0,331.0,71.0,12.76,38.64,26.81,24.0,31.0,39.0,906.37,671.0,769.17,-253.0,9.66,10.62,10.26,77.61,81.97,78.88,-314.72,-48.54,-27.06,-75.12,-10.90,-1

In [9]:
def getX(gameStats, filters):
    return gameStats[filters]
def getY(gameStats):
    return gameStats["HomeWin"]

## Training Model

In [10]:
filter_base = ["GS","GA","GD","Win%","xGoalsPercentage","corsiPercentage","fenwickPercentage","shotAttemptsFor","shotAttemptsAgainst","xReboundsFor","xReboundsAgainst","xOnGoalFor","xOnGoalAgainst","xGoalsFor","xGoalsAgainst","xReboundsFor","xReboundsAgainst","shotsOnGoalFor","shotsOnGoalAgainst","missedShotsFor","missedShotsAgainst","blockedShotAttemptsFor","blockedShotAttemptsAgainst","takeawaysFor","takeawaysAgainst"]
x_base = getX(all_game_stats, filter_base)
y_base = getY(all_game_stats)
x_train_base, x_test_base, y_train_base, y_test_base = train_test_split(x_base.values, y_base.values, test_size=0.25, random_state=2)
x_base

,GS,GA,GD,Win%,xGoalsPercentage,corsiPercentage,fenwickPercentage,shotAttemptsFor,shotAttemptsAgainst,xReboundsFor,xReboundsAgainst,xOnGoalFor,xOnGoalAgainst,xGoalsFor,xGoalsAgainst,xReboundsFor,xReboundsAgainst,shotsOnGoalFor,shotsOnGoalAgainst,missedShotsFor,missedShotsAgainst,blockedShotAttemptsFor,blockedShotAttemptsAgainst,takeawaysFor,takeawaysAgainst
0,15.0,4.0,11.0,0.060976,-0.01,-0.01,-0.02,-343.0,-111.0,-5.49,3.60,-168.23,-53.08,-3.21,12.50,-5.49,3.60,-315.0,-135.0,67.0,74.0,-95.0,-50.0,-81.0,-35.0
1,50.0,-37.0,87.0,0.195122,0.04,0.01,0.03,636.0,343.0,26.47,6.04,382.29,50.43,52.96,9.21,26.47,6.04,449.0,92.0,57.0,-28.0,130.0,279.0,160.0,153.0
2,94.0,-66.0,160.0,0.390244,0.12,0.07,0.08,751.0,-636.0,43.15,-27.06,501.11,-314.72,78.21,-48.54,43.15,-27.06,391.0,-312.0,280.0,-112.0,80.0,-212.0,229.0,174.0
3,21.0,38.0,-17.0,-0.097561,0.01,0.03,0.04,395.0,-178.0,14.05,-7.34,213.09,-120.11,6.02,-1.34,14.05,-7.34,186.0,-149.0,115.0,-12.0,94.0,-17.0,-75.0,57.0
4,93.0,-57.0,150.0,0.341463,0.06,0.07,0.06,892.0,-328.0,39.91,0.63,508.24,-56.90,71.93,12.65,39.91,0.63,512.0,-76.0,176.0,-36.0,204.0,-216.0,-1.0,-62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4354,20.0,-5.0,25.0,0.156404,0.09,0.03,0.02,141.0,-40.0,9.53,-1.32,77.55,-5.66,26.62,-6.75,9.53,-1.32,102.0,5.0,-28.0,-14.0,67.0,-31.0,-13.0,29.0
4355,1.0,-36.0,37.0,0.167143,0.02,-0.01,0.00,-250.0,-188.0,-9.15,-2.57,-92.73,-76.95,-4.52,-11.60,-9.15,-2.57,-73.0,-113.0,-58.0,-5.0,-119.0,-70.0,-14.0,-33.0
4356,-6.0,-16.0,10.0,0.136923,0.01,-0.02,-0.03,-215.0,-110.0,-8.51,-3.01,-125.58,-30.81,-8.02,-12.15,-8.51,-3.01,-135.0,-33.0,-27.0,-11.0,-53.0,-66.0,-4.0,-29.0
4357,13.0,15.0,-2.0,0.027473,-0.03,-0.07,-0.04,-51.0,371.0,0.04,9.33,8.03,150.91,1.90,12.75,0.04,9.33,49.0,99.0,-38.0,121.0,-62.0,151.0,4.0,11.0


In [11]:
knn = KNN(500)
knn.fit(x_train_base, y_train_base)

In [12]:
pred_y_train_base = knn.predict_loop(x_train_base)
print("Training Accuracy ", accuracy_score(pred_y_train_base,y_train_base))

Training Accuracy  0.9247476292444172


In [13]:
pred_y_test_base = knn.predict_loop(x_test_base)
print("Testing Accuracy  ", accuracy_score(pred_y_test_base,y_test_base))

Testing Accuracy   0.5706422018348624


In [14]:
def getTeam(team, stats):
    return stats.loc[stats["team"] == team]

def getGameSet(awayTeam, homeTeam):
    awayTeam = awayTeam.drop(["team"], axis=1)
    homeTeam = homeTeam.drop(["team"], axis=1)
    diff = pd.DataFrame(np.subtract(homeTeam.values, awayTeam.values), columns=homeTeam.columns)
    return diff
    
away = getTeam("Carolina Hurricanes", teamdata25)
home = getTeam("Pittsburgh Penguins", teamdata25)
gameSet = getGameSet(away, home)    
print(knn.predict_loop(getX(gameSet, filter_base).values))
# getX(gameSet, filter)

[0]


## Optimized Baseline

In [15]:
all_game_stats

,Away,AwayScore,Home,HomeScore,HomeWin,GS,GA,GD,Win%,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst
0,Pittsburgh Penguins,6.0,Tampa Bay Lightning,2.0,0,15.0,4.0,11.0,0.060976,0.0,-0.01,-0.01,-0.02,-272.0,-168.23,-3.21,-5.49,-62.17,-5.91,-85.68,-85.55,0.15,-3.24,0.11,-315.0,67.0,-95.0,-343.0,16.0,-63.0,-5.0,10.0,15.0,-78.0,-149.0,-331.0,-264.0,86.0,223.0,-21.0,114.0,-81.0,-186.0,-307.0,61.0,-2.0,-6.65,7.54,-4.09,10.0,-6.0,12.0,-350.10,-248.0,-254.12,-45.0,-0.70,-11.17,-12.03,8.12,7.68,7.84,-53.08,12.50,3.60,-36.59,-3.81,-3.49,-33.20,12.30,12.60,12.40,-135.0,74.0,-50.0,-111.0,6.0,-52.0,9.0,-29.0,10.0,-51.0,55.0,-141.0,-67.0,57.0,183.0,71.0,-119.0,-35.0,-121.0,-146.0,87.0,-2.0,1.78,10.97,-0.26,-4.0,1.0,9.0,-112.61,-61.0,-59.75,-28.0,0.99,-8.72,-8.35,21.83,21.78,20.56
1,Seattle Kraken,3.0,Vegas Golden Knights,4.0,1,50.0,-37.0,87.0,0.195122,0.0,0.04,0.01,0.03,1796.0,382.29,52.96,26.47,95.61,11.89,166.38,152.71,47.87,55.00,49.86,449.0,57.0,130.0,636.0,49.0,69.0,7.0,84.0,32.0,253.0,19.0,400.0,457.0,-17.0,-60.0,194.0,-91.0,160.0,90.0,291.0,144.0,71.0,8.66,17.78,26.50,2.0,28.0,19.0,715.96,506.0,548.29,18.0,5.72,16.22,16.21,42.46,44.81,42.57,50.43,9.21,6.04,6.00,0.76,23.01,19.00,7.62,6.44,4.94,92.0,-28.0,279.0,343.0,-40.0,15.0,2.0,-117.0,3.0,116.0,87.0,132.0,104.0,-16.0,-49.0,47.0,646.0,153.0,-10.0,-18.0,53.0,29.0,-0.41,8.02,1.58,-4.0,-9.0,-27.0,248.60,64.0,4.58,-37.0,1.53,0.24,0.28,10.46,7.22,6.01
2,Montreal Canadiens,1.0,Toronto Maple Leafs,2.0,1,94.0,-66.0,160.0,0.390244,0.0,0.12,0.07,0.08,-223.0,501.11,78.21,43.15,84.25,14.06,278.77,172.58,73.52,82.36,77.55,391.0,280.0,80.0,751.0,94.0,19.0,9.0,101.0,32.0,271.0,154.0,297.0,577.0,-46.0,-102.0,314.0,-258.0,229.0,-179.0,269.0,331.0,71.0,12.76,38.64,26.81,24.0,31.0,39.0,906.37,671.0,769.17,-253.0,9.66,10.62,10.26,77.61,81.97,78.88,-314.72,-48.54,-27.06,-75.12,-10.90,-1

In [16]:
from sklearn.preprocessing import Normalizer
scaler = Normalizer()
x_base_opt = getX(all_game_stats, filter_base)
x_base_opt = pd.DataFrame(scaler.fit_transform(x_base_opt.T).T,columns=x_base_opt.columns)
y_base_opt = getY(all_game_stats)
x_train_base_opt, x_test_base_opt, y_train_base_opt, y_test_base_opt = train_test_split(x_base_opt.values, y_base_opt.values, test_size=0.25, random_state=25)
x_base_opt


,GS,GA,GD,Win%,xGoalsPercentage,corsiPercentage,fenwickPercentage,shotAttemptsFor,shotAttemptsAgainst,xReboundsFor,xReboundsAgainst,xOnGoalFor,xOnGoalAgainst,xGoalsFor,xGoalsAgainst,xReboundsFor,xReboundsAgainst,shotsOnGoalFor,shotsOnGoalAgainst,missedShotsFor,missedShotsAgainst,blockedShotAttemptsFor,blockedShotAttemptsAgainst,takeawaysFor,takeawaysAgainst
0,0.005196,0.001308,0.002121,0.005410,-0.002589,-0.003124,-0.006207,-0.010106,-0.003666,-0.003948,0.002907,-0.008974,-0.003399,-0.001279,0.006109,-0.003948,0.002907,-0.016825,-0.008191,0.007333,0.010078,-0.009413,-0.004084,-0.009993,-0.004287
1,0.017321,-0.012095,0.016772,0.017313,0.010357,0.003124,0.009311,0.018739,0.011328,0.019033,0.004878,0.020393,0.003230,0.021095,0.004501,0.019033,0.004878,0.023982,0.005582,0.006238,-0.003813,0.012881,0.022790,0.019740,0.018739
2,0.032564,-0.021574,0.030846,0.034626,0.031070,0.021868,0.024830,0.022127,-0.021004,0.031027,-0.021853,0.026732,-0.020155,0.031153,-0.023723,0.031027,-0.021853,0.020884,-0.018931,0.030644,-0.015253,0.007927,-0.017317,0.028253,0.021312
3,0.007275,0.012421,-0.003277,-0.008657,0.002589,0.009372,0.012415,0.011638,-0.005878,0.010103,-0.005928,0.011367,-0.007692,0.002398,-0.000655,0.010103,-0.005928,0.009935,-0.009041,0.012586,-0.001634,0.009314,-0.001389,-0.009253,0.006981
4,0.032218,-0.018632,0.028918,0.030298,0.015535,0.021868,0.018622,0.026281,-0.010832,0.028697,0.000509,0.027112,-0.003644,0.028651,0.006182,0.028697,0.000509,0.027347,-0.004611,0.019262,-0.004903,0.020214,-0.017644,-0.000123,-0.007594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4354,0.006929,-0.001634,0.004820,0.013878,0.023302,0.009372,0.006207,0.004154,-0.001321,0.006853,-0.001066,0.004137,-0.000362,0.010603,-0.003299,0.006853,-0.001066,0.005448,0.000303,-0.003064,-0.001907,0.006639,-0.002532,-0.001604,0.003552
4355,0.000346,-0.011768,0.007133,0.014831,0.005178,-0.003124,0.000000,-0.007366,-0.006209,-0.006579,-0.002075,-0.004947,-0.004928,-0.001800,-0.005669,-0.006579,-0.002075,-0.003899,-0.006857,-0.006348,-0.000681,-0.011791,-0.005718,-0.001727,-0.004042
4356,-0.002079,-0.005230,0.001928,0.012149,0.002589,-0.006248,-0.009311,-0.006335,-0.003633,-0.006119,-0.002431,-0.006699,-0.001973,-0.003195,-0.005938,-0.006119,-0.002431,-0.007211,-0.002002,-0.002955,-0.001498,-0.005252,-0.005391,-0.000494,-0.003552
4357,0.004504,0.004903,-0.000386,0.002438,-0.007767,-0.021868,-0.012415,-0.001503,0.012252,0.000029,0.007535,0.000428,0.009664,0.000757,0.006231,0.000029,0.007535,0.002617,0.006007,-0.004159,0.016479,-0.006143,0.012334,0.000494,0.001347


In [17]:
knn_opt = KNN(1000)
knn_opt.fit(x_train_base_opt, y_train_base_opt)

In [18]:
pred_y_train_base_opt = knn_opt.predict_loop(x_train_base_opt)
print("Training Accuracy ", accuracy_score(pred_y_train_base_opt,y_train_base_opt))

KeyboardInterrupt: 

In [ ]:
knn_opt = KNN(1000)
knn_opt.fit(x_train_base_opt, y_train_base_opt)
pred_y_test_base_opt = knn_opt.predict_loop(x_test_base_opt)
print("Testing Accuracy  ", accuracy_score(pred_y_test_base_opt,y_test_base_opt))

Testing Accuracy   0.5854108956602031


In [ ]:
knn_opt = KNN(500)
knn_opt.fit(x_train_base_opt, y_train_base_opt)
pred_y_test_base_opt = knn_opt.predict_loop(x_test_base_opt)
print("Testing Accuracy  ", accuracy_score(pred_y_test_base_opt,y_test_base_opt))

Testing Accuracy   0.5844875346260388


In [ ]:
knn_opt = KNN(200)
knn_opt.fit(x_train_base_opt, y_train_base_opt)
pred_y_test_base_opt = knn_opt.predict_loop(x_test_base_opt)
print("Testing Accuracy  ", accuracy_score(pred_y_test_base_opt,y_test_base_opt))

In [19]:
knn_opt = KNN(100)
knn_opt.fit(x_train_base_opt, y_train_base_opt)
pred_y_test_base_opt = knn_opt.predict_loop(x_test_base_opt)
print("Testing Accuracy  ", accuracy_score(pred_y_test_base_opt,y_test_base_opt))

Testing Accuracy   0.5972477064220183


In [ ]:
def getTeam(team, stats):
    return stats.loc[stats["team"] == team]

def getGameSet(awayTeam, homeTeam):
    awayTeam = awayTeam.drop(["team"], axis=1)
    homeTeam = homeTeam.drop(["team"], axis=1)
    diff = pd.DataFrame(np.subtract(homeTeam.values, awayTeam.values), columns=homeTeam.columns)
    return diff
    
away = getTeam("Carolina Hurricanes", teamdata25)
home = getTeam("Pittsburgh Penguins", teamdata25)
gameSet = getGameSet(away, home)
prediction = getX(gameSet, filter_base)
prediction = pd.DataFrame(scale(prediction.T).T,columns=prediction.columns)
print(knn_opt.predict_loop(prediction.values))
# getX(gameSet, filter)
prediction

NameError: name 'teamdata25' is not defined